## StateGraph
> A graph whose nodes communicate by reading and writing to a shared state. The signature of each node is State -> Partial.

* Each State key can be annotated with a reducer function that will be used to aggregate (combine all) the values of that key received from multiple nodes. The Signature of a reducer function is (Value, Value) -> Value.

* Parameters:
    * state_schema (Type[Any], default = None) -> The Schema class that define that state.
    * config_schema (Optional[Type[Any]], default = None) - The schema class that define the configuration Use this to expose configurable parameters in you API.

**Examples ⏬** 

In [1]:
from langchain_core.runnables import RunnableConfig
from typing_extensions import Annotated, TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph

def reducer(a: list, b: int | None) -> list:
    print("A:: ",a,"\nB::",b)

    if b is not None:
        return a + [b]
    
    return a

class State(TypedDict):
    x : Annotated[list, reducer]

class ConfigSchema(TypedDict):
    r : float

graph = StateGraph(State, config_schema = ConfigSchema)

def node(state : State, config: RunnableConfig) -> dict:
    print("STATE:: ",state)

    r = config["configurable"].get("r", 1.0)
    x = state["x"][-1]
    next_value = x * r * (1 - x)
    return {"x" : next_value}

graph.add_node("A", node)
graph.set_entry_point("A") # also like --> graph.add_node(START, "A") -> the first node in graph-type pregel.
graph.set_finish_point("A") # also like --> graph.add_node(END, "A") -> The last (maybe virtual) node in graph-style Pregel.el.

compiled = graph.compile()

# print(compiled.config_specs)

step1 = compiled.invoke({"x" : 0.5}, {"configurable": {"r": 3.0}})
print("Value is:: ",step1)

# step2 = compiled.invoke({"x" : 1}, {"configurable": {"r": 3.0}})
# print(step2)

A::  [] 
B:: 0.5
STATE::  {'x': [0.5]}
A::  [0.5] 
B:: 0.75
Value is::  {'x': [0.5, 0.75]}
